In [21]:
import pandas as pd
import numpy as np


In [22]:
path_train = r"train"
path_test = r"test"

In [23]:
# данные по дислокации
dislok_train = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
dislok_test = pd.read_parquet(path_test + '/dislok_wagons.parquet').convert_dtypes()
# данные по текущим ремонтам
pr_rem_train = pd.read_parquet(path_train + '/pr_rems.parquet').convert_dtypes()
pr_rem_test = pd.read_parquet(path_test + '/pr_rems.parquet').convert_dtypes()
# список вагонов с остаточным пробегом на момент прогноза
wag_prob_train = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
wag_prob_test = pd.read_parquet(path_test + '/wagons_probeg_ownersip.parquet').convert_dtypes()
 # параметры вагона
wag_param_train = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
wag_param_test = pd.read_parquet(path_test + '/wag_params.parquet').convert_dtypes()
 # текущие ремонты вагонов
tr_rem_train = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()
tr_rem_test = pd.read_parquet(path_test + '/tr_rems.parquet').convert_dtypes()
 # справочник грузов
fr_info_train = pd.read_parquet(path_train + '/freight_info.parquet').convert_dtypes()
fr_info_test = pd.read_parquet(path_test + '/freight_info.parquet').convert_dtypes()
 # данные по КТИ
kti_izm_train = pd.read_parquet(path_train + '/kti_izm.parquet').convert_dtypes()
kti_izm_test = pd.read_parquet(path_test + '/kti_izm.parquet').convert_dtypes()
 # справочник станций
stations_train = pd.read_parquet(path_train + '/stations.parquet').convert_dtypes()
stations_test = pd.read_parquet(path_test + '/stations.parquet').convert_dtypes()
 # таргет по прогноза выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()

In [24]:
dislok = pd.concat([dislok_train, dislok_test]).drop_duplicates()
pr_rem = pd.concat([pr_rem_train, pr_rem_test]).drop_duplicates()
wag_prob = pd.concat([wag_prob_train, wag_prob_test]).drop_duplicates()
wag_param = pd.concat([wag_param_train, wag_param_test]).drop_duplicates()
tr_rem = pd.concat([tr_rem_train, tr_rem_test]).drop_duplicates()
fr_info = pd.concat([fr_info_train, fr_info_test]).drop_duplicates()
kti_izm = pd.concat([kti_izm_train, kti_izm_test]).drop_duplicates()
stations = pd.concat([stations_train, stations_test]).drop_duplicates()

In [25]:
# Оставляю только глазами отобранные данные
wag_prob = wag_prob[[
                'repdate', 
                'wagnum', 
                'ost_prob'
                     ]]
wag_prob.repdate = pd.to_datetime(wag_prob.repdate)
wag_prob['repdate_month'] = wag_prob.repdate.apply(lambda x: x.strftime('%Y-%m'))


dislok = dislok[[
            'plan_date',
            'wagnum',
            'date_kap',
            'date_dep',
            'kod_vrab',
            'date_pl_rem',
            'ost_prob'
             ]]


wag_param = wag_param[[
                    'wagnum',
                    'cnsi_gruz_capacity',
                    'cnsi_volumek',
                    'tara',
                    'date_build',
                    'srok_sl',
                    'cnsi_probeg_dr',
                    'cnsi_probeg_kr'
                    ]]


pr_rem = pr_rem[[
                'wagnum',
                'rem_month'
                ]]


tr_rem = tr_rem[[
            'wagnum',
            'rem_month',
            'gr_probeg',
            'por_probeg'
                ]]
tr_rem = tr_rem.drop_duplicates()

In [26]:
# Номера вагонов без пропусков в пробеге
prob_valid_wagnums = wag_prob[wag_prob.ost_prob.isna()].wagnum.drop_duplicates()

In [27]:
# Пробеги только тех, у кого нет пропусков
wag_prob_valid = wag_prob[~wag_prob.wagnum.isin(prob_valid_wagnums)]

In [28]:
# Создание датафрейма со средним количеством пробега за прошлый месяц

wag_prob_valid['min_mnthly_date'] = wag_prob_valid.groupby(['wagnum', 'repdate_month'])['repdate'].transform('min')
wag_prob_valid['max_mnthly_date'] = wag_prob_valid.groupby(['wagnum', 'repdate_month'])['repdate'].transform('max')

min_wag_prob = wag_prob_valid[wag_prob_valid.min_mnthly_date == wag_prob_valid.repdate][['repdate_month', 'wagnum', 'ost_prob']]\
    .rename(columns={'ost_prob':'max_mnthly_prob'})
max_wag_prob = wag_prob_valid[wag_prob_valid.max_mnthly_date == wag_prob_valid.repdate][['repdate_month', 'wagnum', 'ost_prob']]\
    .rename(columns={'ost_prob':'min_mnthly_prob'})

wag_prob_valid['days'] = (wag_prob_valid.max_mnthly_date - wag_prob_valid.min_mnthly_date).apply(lambda x: x.days)
mounthly_recording_days = wag_prob_valid[['repdate_month', 'wagnum', 'days']].drop_duplicates()

mounthly_prob_spending = min_wag_prob.merge(max_wag_prob, on=['repdate_month', 'wagnum'])\
                                    .merge(mounthly_recording_days, on=['repdate_month', 'wagnum'])

mounthly_prob_spending = mounthly_prob_spending.merge(wag_param[['wagnum', 'cnsi_probeg_dr', 'cnsi_probeg_kr']], on='wagnum')


mounthly_prob_spending.cnsi_probeg_dr = mounthly_prob_spending.cnsi_probeg_dr * 1000
mounthly_prob_spending.cnsi_probeg_kr = mounthly_prob_spending.cnsi_probeg_kr * 1000

C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\367530856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wag_prob_valid['min_mnthly_date'] = wag_prob_valid.groupby(['wagnum', 'repdate_month'])['repdate'].transform('min')
C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\367530856.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wag_prob_valid['max_mnthly_date'] = wag_prob_valid.groupby(['wagnum', 'repdate_month'])['repdate'].transform('max')
C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\367530

In [29]:
# Костыль, чтобы не париться с логикой правильного выбора cnsi_probeg_dr или cnsi_probeg_kr
def closer_number(num1, num2, target):
    distance_to_num1 = abs(target - num1)
    distance_to_num2 = abs(target - num2)

    if distance_to_num1 < distance_to_num2:
        return num1
    else:
        return num2
    


def daily_avg_probeg(max_mnthly_prob, min_mnthly_prob, days, cnsi_probeg_dr, cnsi_probeg_kr):
    if max_mnthly_prob > min_mnthly_prob:
        return (max_mnthly_prob - min_mnthly_prob)/days
    else:
        return (max_mnthly_prob - min_mnthly_prob + closer_number(cnsi_probeg_dr, cnsi_probeg_kr, min_mnthly_prob))/days

In [30]:
mounthly_prob_spending['daily_avg_probeg_per_mnth'] = \
    mounthly_prob_spending.apply(lambda x: daily_avg_probeg(
                                                            x.max_mnthly_prob,
                                                            x.min_mnthly_prob,
                                                            x.days,
                                                            x.cnsi_probeg_dr,
                                                            x.cnsi_probeg_kr
                                                        ), axis=1)


mounthly_prob_spending = mounthly_prob_spending[['repdate_month', 'wagnum', 'daily_avg_probeg_per_mnth']]

mounthly_prob_spending['rolling_daily_avg'] = \
        mounthly_prob_spending.groupby('wagnum')['daily_avg_probeg_per_mnth']\
        .transform(lambda x: x.rolling(10, 1).mean())

In [31]:
from dateutil import relativedelta
from datetime import datetime

In [32]:
def plus_mounth(input_date):
        # Преобразование строки в объект Timestamp
    timestamp = pd.to_datetime(input_date)

    # Получение следующего месяца
    next_month = timestamp + pd.DateOffset(months=1)

    return next_month

In [33]:
mounthly_prob_spending['target_month'] = mounthly_prob_spending.repdate_month.apply(lambda x: plus_mounth(x))

In [34]:
# Новая колонка - оставшийся пробег на последнюю дату предыдущего месяца
mounthly_prob_spending = mounthly_prob_spending.merge(max_wag_prob, on=['repdate_month', 'wagnum'])\
                        .rename(columns={'min_mnthly_prob':'last_prob'})

In [35]:
mounthly_prob_spending = mounthly_prob_spending[[
                                            'target_month',
                                            'wagnum',
                                            'last_prob',
                                            'daily_avg_probeg_per_mnth',
                                            'rolling_daily_avg',
                                            'repdate_month'
                                        ]]

In [36]:
mounthly_prob_spending

,target_month,wagnum,last_prob,daily_avg_probeg_per_mnth,rolling_daily_avg,repdate_month
0,2022-09-01,33361,4105,114.533333,114.533333,2022-08
1,2022-10-01,33361,2185,66.206897,90.370115,2022-09
2,2022-11-01,33361,1236,31.633333,70.791188,2022-10
3,2022-12-01,33361,159916,45.517241,64.472701,2022-11
4,2023-01-01,33361,153692,207.466667,93.071494,2022-12
...,...,...,...,...,...,...
225252,2023-01-01,8061,149388,250.233333,490.140445,2022-12
225253,2023-02-01,8061,140004,312.800000,454.672356,2023-01
225254,2023-03-01,8061,139890,3.703704,379.510914,2023-02
225255,2022-12-01,33350,90841,15625.000000,15625.000000,2022-11


In [37]:
dislok.plan_date = pd.to_datetime(dislok.plan_date)
dislok.date_dep = pd.to_datetime(dislok.date_dep)
dislok.date_pl_rem = pd.to_datetime(dislok.date_pl_rem)

In [38]:
dislok_start_mnth = dislok[dislok['plan_date'].dt.is_month_start]

In [39]:
def date_pl_rem_valid_checker(date_pl_rem, plan_date):
    if date_pl_rem < plan_date:
        return None
    else:
        return date_pl_rem

In [40]:
dislok_start_mnth.date_pl_rem = dislok_start_mnth.apply(lambda x: date_pl_rem_valid_checker(x.date_pl_rem, x.plan_date), axis=1)
dislok_start_mnth.kod_vrab = dislok_start_mnth.apply(lambda x: None if pd.isna(x.date_pl_rem) else x.kod_vrab, axis=1)

C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\801395818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dislok_start_mnth.date_pl_rem = dislok_start_mnth.apply(lambda x: date_pl_rem_valid_checker(x.date_pl_rem, x.plan_date), axis=1)
C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\801395818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dislok_start_mnth.kod_vrab = dislok_start_mnth.apply(lambda x: None if pd.isna(x.date_pl_rem) else x.kod_vrab, axis=1)


In [41]:
def days_from_kap_rem(date_kap, plan_date):
    if pd.notna(date_kap):
        return (plan_date - date_kap).days
    else:
        return None

In [42]:
dislok_start_mnth['days_from_kap_rem'] = dislok_start_mnth.apply(lambda x: days_from_kap_rem(x.date_kap, x.plan_date), axis=1)

C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\1262972826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dislok_start_mnth['days_from_kap_rem'] = dislok_start_mnth.apply(lambda x: days_from_kap_rem(x.date_kap, x.plan_date), axis=1)


In [43]:
def days_from_dep_rem(date_dep, plan_date):
    if pd.notna(date_dep):
        return (plan_date - date_dep).days
    else:
        return None

In [44]:
dislok_start_mnth['days_from_dep_rem'] = dislok_start_mnth.apply(lambda x: days_from_kap_rem(x.date_dep, x.plan_date), axis=1)

C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\1995753316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dislok_start_mnth['days_from_dep_rem'] = dislok_start_mnth.apply(lambda x: days_from_kap_rem(x.date_dep, x.plan_date), axis=1)


In [45]:
def days_to_plan_rem(date_pl_rem, plan_date):
    if pd.notna(date_pl_rem):
        return (date_pl_rem - plan_date).days
    else:
        return None

In [46]:
dislok_start_mnth['days_to_plan_rem'] = dislok_start_mnth.apply(lambda x: days_to_plan_rem(x.date_pl_rem, x.plan_date), axis=1)

C:\Users\Renat\AppData\Local\Temp\ipykernel_14176\2737801142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dislok_start_mnth['days_to_plan_rem'] = dislok_start_mnth.apply(lambda x: days_to_plan_rem(x.date_pl_rem, x.plan_date), axis=1)


In [47]:
dislok_start_mnth = dislok_start_mnth[[
                            'wagnum',
                            'plan_date',
                            'ost_prob',
                            'days_from_kap_rem',
                            'days_from_dep_rem',
                            'days_to_plan_rem',
                            'kod_vrab'
                        ]]

In [48]:
wag_param.date_build = pd.to_datetime(wag_param.date_build)
wag_param.srok_sl = pd.to_datetime(wag_param.srok_sl)
wag_param.cnsi_probeg_kr = wag_param.cnsi_probeg_kr.apply(lambda x: 160 if x == 0 else x)

In [49]:
pr_rem[pr_rem.wagnum == 28268]

,wagnum,rem_month
215,28268,2023-01-05
360,28268,2022-12-30


In [50]:
pr_rem

,wagnum,rem_month
0,15000,2022-08-05
2,25485,2022-08-21
3,25944,2022-08-09
4,24080,2022-08-13
5,28612,2022-08-09
...,...,...
1883,5049,2023-02-28
1884,15915,2023-02-28
1890,7276,2023-02-24
1895,25459,2023-02-02


In [51]:
tr_rem

,wagnum,rem_month,gr_probeg,por_probeg
0,29938,2022-08-01,87913,37467
2,29852,2022-08-01,81824,45579
4,13674,2022-08-01,64130,34697
6,1145,2022-08-01,89735,39549
8,2430,2022-08-01,68986,26528
...,...,...,...,...
7493,19485,2023-02-01,83707,33694
7495,5300,2023-02-01,23945,10147
7496,13550,2023-02-01,92068,38886
7497,33743,2023-02-01,16408,7664


In [52]:
y_train = pd.read_csv('train/target/y_train.csv')

In [53]:
mounthly_prob_spending.rename(columns={'target_month':'month'}, inplace=True)
mounthly_prob_spending.drop(columns=['repdate_month'], inplace=True)

In [54]:
dislok_start_mnth.rename(columns={'plan_date':'month'}, inplace=True)

In [55]:
pr_rem.sort_values(['wagnum', 'rem_month'], ascending=[True, False], inplace=True)
pr_rem['row_num'] = pr_rem.groupby('wagnum').cumcount() + 1
pr_rem = pr_rem[pr_rem.row_num == 1].drop(columns=['row_num'])

In [56]:
tr_rem.rem_month = pd.to_datetime(tr_rem.rem_month)
tr_rem.sort_values(['wagnum', 'rem_month'], ascending=[True, False], inplace=True)
tr_rem['row_num'] = tr_rem.groupby('wagnum').cumcount() + 1

In [57]:
def days_from_build(date_build, month):
    if pd.notna(date_build):
        return (month - date_build).days
    else:
        return None
    

def days_to_liq(srok_sl, month):
    if pd.notna(srok_sl):
        return (srok_sl - month).days
    else:
        return None

def days_from_pr_rem(rem_month, month):
    if pd.notna(rem_month):
        days = (month - rem_month).days
        if days > 0:
            return days
        else:
            return None
    else:
        return None
    

def info_from_tr_rem(wagnum, month):
    temp = tr_rem[tr_rem.wagnum == wagnum]
    if temp.shape[0] == 0:
        return None
    temp = temp[temp.rem_month < month]
    cnt = temp.shape[0]
    t1 = temp[temp.rem_month == temp.rem_month.max()]
    if t1.shape[0] == 0:
        return None
    rem_month = t1.rem_month.iloc[0]
    gr_probeg = t1.gr_probeg.iloc[0]
    por_probeg = t1.por_probeg.iloc[0]

    return (cnt, rem_month, gr_probeg, por_probeg)


def days_from_last_tr_rem(last_tr_rem_date, month):
    if pd.notna(last_tr_rem_date):
        days = (month - last_tr_rem_date).days
        if days > 0:
            return days
        else:
            return None
    else:
        return None
    

In [58]:
mounthly_prob_spending

,month,wagnum,last_prob,daily_avg_probeg_per_mnth,rolling_daily_avg
0,2022-09-01,33361,4105,114.533333,114.533333
1,2022-10-01,33361,2185,66.206897,90.370115
2,2022-11-01,33361,1236,31.633333,70.791188
3,2022-12-01,33361,159916,45.517241,64.472701
4,2023-01-01,33361,153692,207.466667,93.071494
...,...,...,...,...,...
225252,2023-01-01,8061,149388,250.233333,490.140445
225253,2023-02-01,8061,140004,312.800000,454.672356
225254,2023-03-01,8061,139890,3.703704,379.510914
225255,2022-12-01,33350,90841,15625.000000,15625.000000


In [59]:
def X(y_train):
    "Обогащает X данными"
    y_train.month = pd.to_datetime(y_train.month)
    m1 = y_train.merge(mounthly_prob_spending, on=['wagnum', 'month'], how='left')
    m2 = m1.merge(dislok_start_mnth.drop_duplicates(), on=['wagnum', 'month'], how='left')
    m3 = m2.merge(wag_param, how='left', on='wagnum')
    m3['days_from_build'] = m3.apply(lambda x: days_from_build(x.date_build, x.month), axis=1)
    m3['days_to_liq'] = m3.apply(lambda x: days_to_liq(x.srok_sl, x.month), axis=1)
    m3.drop(columns=['date_build', 'srok_sl'], inplace=True)
    m4 = m3.merge(pr_rem, on='wagnum', how='left')
    m4['days_from_pr_rem'] = m4.apply(lambda x: days_from_pr_rem(x.rem_month, x.month), axis=1)
    m4.drop(columns=['rem_month'], inplace=True)
    m4['dop_info'] = m4.apply(lambda x: info_from_tr_rem(x.wagnum, x.month), axis=1)
    m4['tr_rem_cnt_before'] = m4.dop_info.apply(lambda x: x[0] if pd.notna(x) else x)
    m4['last_tr_rem_date'] = m4.dop_info.apply(lambda x: x[1] if pd.notna(x) else x)
    m4['days_from_last_tr_rem'] = m4.apply(lambda x: days_from_last_tr_rem(x.last_tr_rem_date, x.month), axis=1)
    m4.drop(columns=['dop_info', 'last_tr_rem_date'], inplace=True)
    m4['predict_probeg_in_10_days'] = (m4.ost_prob - m4.rolling_daily_avg * 10)
    m4['predict_probeg_in_1_mnth'] = (m4.ost_prob - m4.rolling_daily_avg * 30)



    m4['last_prob'] = m4['last_prob'].astype(float).fillna(m4['last_prob'].mean())
    m4['daily_avg_probeg_per_mnth'] = m4['daily_avg_probeg_per_mnth'].astype(float).fillna(m4['daily_avg_probeg_per_mnth'].mean())
    m4['rolling_daily_avg'] = m4['rolling_daily_avg'].astype(float).fillna(m4['rolling_daily_avg'].mean())
    m4['ost_prob'] = m4['ost_prob'].astype(float).fillna(m4['ost_prob'].mean())
    m4['days_from_kap_rem'] = m4['days_from_kap_rem'].astype(float).fillna(m4['days_from_kap_rem'].mean())
    m4['days_from_dep_rem'] = m4['days_from_dep_rem'].astype(float).fillna(m4['days_from_dep_rem'].mean())
    m4['days_to_plan_rem'] = m4['days_to_plan_rem'].astype(float).fillna(m4['days_to_plan_rem'].mean())
    m4['kod_vrab'] = m4['kod_vrab'].astype(float).fillna(m4['kod_vrab'].mean())
    m4['days_from_pr_rem'] = m4['days_from_pr_rem'].astype(float).fillna(m4['days_from_pr_rem'].mean())
    m4['tr_rem_cnt_before'] = m4['tr_rem_cnt_before'].astype(float).fillna(m4['tr_rem_cnt_before'].mean())
    m4['days_from_last_tr_rem'] = m4['days_from_last_tr_rem'].astype(float).fillna(m4['days_from_last_tr_rem'].mean())

    m4['predict_probeg_in_10_days'] = m4['predict_probeg_in_10_days'].astype(float).fillna(m4['predict_probeg_in_10_days'].mean())
    m4['predict_probeg_in_1_mnth'] = m4['predict_probeg_in_1_mnth'].astype(float).fillna(m4['predict_probeg_in_1_mnth'].mean())

    m4['predict_probeg_in_10_days'] = m4['predict_probeg_in_10_days'].astype(float).fillna(m4['predict_probeg_in_10_days'].mean())
    m4['predict_probeg_in_1_mnth'] = m4['predict_probeg_in_1_mnth'].astype(float).fillna(m4['predict_probeg_in_1_mnth'].mean())

    #X = m4.drop(columns=['wagnum', 'month', 'target_month', 'target_day']).values

    #y_mnth = m4.target_month.values

    #y_d = m4.target_day.values

    return m4

In [73]:
y_predict = pd.read_csv('train/target/y_predict.csv')

In [75]:
predict_df = X(y_predict)

In [82]:
predict_df.to_csv('predict_df.csv')

In [66]:
train_df = X(y_train)

In [68]:
train_df.to_csv('train_df.csv')

In [ ]:
X = valid_df.drop(columns=['wagnum', 'month', 'target_month', 'target_day']).values

y_mnth = valid_df.target_month.values

y_d = valid_df.target_day.values

 <!-- Первая пристрелочная модель - Логистическая регрессия -->

In [ ]:
# from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import make_scorer, f1_score
# from sklearn.pipeline import Pipeline


In [ ]:
# scaler = StandardScaler()

# logreg = LogisticRegression()

# pipeline = Pipeline([
#     ('scaler', scaler),
#     ('classifier', logreg)
# ])

# param_grid = {
#     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# }

# cv = StratifiedKFold(n_splits=5)

# f1_scorer = make_scorer(f1_score)


# grid_search = GridSearchCV(
#     estimator=pipeline,
#     param_grid=param_grid,
#     scoring=f1_scorer,
#     cv=cv,
#     verbose=1
# )

In [ ]:
# param_grid_for_LogReg = {"C": np.arange(0, 0.5, 0.001)}
# cv = StratifiedKFold(n_splits=5)
# LogReg = RandomizedSearchCV(estimator=LogisticRegression(random_state=17), scoring='f1', param_distributions=param_grid_for_LogReg, cv=cv,  n_jobs=-1)
# LogReg.fit(X, y_d)

C:\Users\Renat\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=LogisticRegression(random_state=17), n_jobs=-1,
                   param_distributions={'C': array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
       0.027, 0.028, 0.029, 0.03 ,...
       0.441, 0.442, 0.443, 0.444, 0.445, 0.446, 0.447, 0.448, 0.449,
       0.45 , 0.451, 0.452, 0.453, 0.454, 0.455, 0.456, 0.457, 0.458,
       0.459, 0.46 , 0.461, 0.462, 0.463, 0.464, 0.465, 0.466, 0.467,
       0.468, 0.469, 0.47 , 0.471, 0.472, 0.473, 0.474, 0.475, 0.476,
       0.477, 0.478, 0.479, 0.48 , 0.481, 0.482, 0.483, 0.484, 0.485,
       0.486, 0.487, 0.488, 0.489, 0.49 , 0.491, 0.492, 0.493, 0.494,
       0.495, 0.496, 0.497, 0.498, 0.499])},
                   scoring='f1')